### 이전 대화를 기억하는 Chain 생성방법

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


### 1. 일반 Chain 에 대화기록 추가

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# llm 생성
llm = ChatOpenAI()

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

In [4]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [5]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "나의 이름은 테디입니다."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'안녕하세요, 테디님! 무엇을 도와드릴까요?'

In [6]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름이 뭐라고?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'당신의 이름은 테디입니다.'

### 2. RAG + RunnableWithMessageHistory

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
import fitz  # PyMuPDF
from langchain_core.document_loaders.base import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# 단계 1: 문서 로드(Load Documents)

class CustomPDFLoader:
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self):
        documents = []
        with fitz.open(self.file_path) as doc:
            for page in doc:
                text = page.get_text()
                documents.append(Document(page_content=text))
        return documents

# 문서를 로드하고, 청크로 나누고, 인덱싱합니다.
# from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
file_path = "data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf"
# loader = PyPDFLoader(file_path=file_path)
loader = CustomPDFLoader("data/1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf")

docs = loader.load()
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = text_splitter.split_documents(docs)

# 단계 3, 4: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings())

# 단계 5: 리트리버 생성(Create Retriever)
# 사용자의 질문(query) 에 부합하는 문서를 검색합니다.

# 유사도 높은 K 개의 문서를 검색합니다.
k = 3

# (Sparse) bm25 retriever and (Dense) faiss retriever 를 초기화 합니다.
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = k

faiss_vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": k})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# 단계 7: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 8: 체인 생성(Create Chain)
chain = (
    {
        "context": itemgetter("question") | ensemble_retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [8]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [9]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "서경의 중요 내용은 무엇이지?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


'《서경》의 중요 내용은 다음과 같습니다:\n\n1. **혁명사상**: 군주가 잘못하면 언제든지 바뀔 수 있음을 천명하는 사상.\n2. **위민사상**: 백성을 편히 살게 하는 사상으로, 서경의 기본 사상.\n3. **덕치사상**: 임금은 어진 자를 등용하고 이들로 하여금 백성들에게 고르게 덕이 미치도록 해야 한다는 사상.\n4. **효사상**: 부모에 대한 효도를 강조하는 사상으로, 도덕의 중심 사상.\n\n또한, 《서경》은 나라의 법도, 임금의 치적, 나라를 다스리는 훈계, 전쟁을 앞두고 임금이 백성과 군사들에게 한 훈시 등을 기록하고 있습니다.'

In [10]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "이전 답변을 영어로 번역해주세요."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


'The important contents of the "Book of Documents" (《서경》) are as follows:\n\n1. **Revolutionary Ideology**: The idea that a ruler can be replaced if they do wrong.\n2. **People-oriented Ideology**: The idea of making the people live comfortably, which is the basic ideology of the "Book of Documents."\n3. **Virtuous Governance Ideology**: The idea that a king should appoint virtuous people and ensure that their virtue reaches the people evenly.\n4. **Filial Piety Ideology**: The idea of emphasizing filial piety towards parents, which is the central moral ideology.\n\nAdditionally, the "Book of Documents" records the laws of the country, the achievements of kings, admonitions for governing the country, and the king\'s instructions to the people and soldiers before a war.'